In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time 

In [ ]:
entidades = pd.read_excel('cod_entidades.xlsx')
cod_entidades = entidades['codigo_chip']
# Muesta pequeña para el ejemplo
cod_entidad = cod_entidades.head(2)
cod_entidad

In [ ]:
# Total de entidades
len(cod_entidades)

In [ ]:
# Creación de un objeto Service, es decir, descarga el driver de Chrome si no está instalado
path = "chromedriver.exe"
service = Service(path)

# Creación de opciones para el navegador Chrome, para asi configurar diferentes aspectos del navegador 
options = webdriver.ChromeOptions()

In [ ]:
columnas_de_interes = ['CODIGO', 'NOMBRE', 'RECAUDO VIGEN ACTUAL SIN FONDOS(Pesos)', 'RECAUDO VIGEN ACTUAL CON FONDOS(Pesos)', 'RECAUDO VIGEN ANTERIOR SIN FONDO(Pesos)', 'RECAUDO VIGEN ANTERIOR CON FONDO(Pesos)', 'TOTAL RECAUDO(Pesos)']
columnas_final = ['CODIGO', 'NOMBRE', 'RECAUDO VIGEN ACTUAL SIN FONDOS(Pesos)', 'RECAUDO VIGEN ACTUAL CON FONDOS(Pesos)', 'RECAUDO VIGEN ANTERIOR SIN FONDO(Pesos)', 'RECAUDO VIGEN ANTERIOR CON FONDO(Pesos)', 'TOTAL RECAUDO(Pesos)']

In [ ]:
# Donde se almacenara todas las filas de las entidades por ciclo
lista_filas = []

In [ ]:
# Creamos nuestro "navegador"
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.chip.gov.co/schip_rt/index.jsf")

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#j_idt105\:InformacionEnviada')))
pag_informacion_ciudadano = driver.find_element(By.CSS_SELECTOR, '#j_idt105\:InformacionEnviada')
pag_informacion_ciudadano.click()

for codigo in cod_entidad:
    while True:
        try:
            # Llenar el filtro de entidad
            filtro_entidad = driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxEntidadCiudadano_input')
            filtro_entidad.clear()
            filtro_entidad.send_keys(codigo)
            filtro_entidad.send_keys(Keys.ENTER)
            break
        except Exception as e:
            print(f"Elemento no encontrado: {e}")
            time.sleep(4)
    
    while True:
        try:
            # Selecciono la opción para el filtro de categorias que es de elección
            filtro_categorias = Select(driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxCategoria'))
            filtro_categorias.select_by_value("K99")
            break
        except Exception as e:
            print(f"Elemento no encontrado: {e}")
            time.sleep(4)
    
    while True:
        try:
            # Selecciono la opción para el filtro de periodo que es de elección
            filtro_periodo = Select(driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxPeriodo'))
            filtro_periodo.select_by_index(1)
            break
        except Exception as e:
            print(f"Elemento no encontrado: {e}")
            time.sleep(4)

    while True:
        try:
            # Selecciono la opción para el filtro de formulario que es de elección
            filtro_formulario = Select(driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxForma'))
            filtro_formulario.select_by_index(2)
            break
        except Exception as e:
            print(f"Elemento no encontrado: {e}")
            time.sleep(4)

    while True:
        try:
            # Ya todos los filtros lleno, le doy al boton generar para generar la tabla
            boton_consultar = driver.find_element(By.CSS_SELECTOR, '#frm1\:BtnConsular')
            boton_consultar.click()
            break
        except Exception as e:
            print(f"Elemento no encontrado: {e}")
            time.sleep(4)
            
    while True:
        try:
            #time.sleep(10)
            # Despúes escojo el filtro de nivel para que me muestre toda la tabla en esa página
            nivel = Select(driver.find_element(By.CSS_SELECTOR, '#frm1\:SelBoxNivel'))
            nivel.select_by_index(8)
            break
        except Exception as e:
            print(f"Elemento no encontrado: {e}")
            time.sleep(4)
        
    while True:
        tabla_html = driver.find_element(By.CSS_SELECTOR, '#frm1\:j_idt260').get_attribute("outerHTML")
        soup = BeautifulSoup(tabla_html, "html.parser")
        filas = soup.find_all('tr')
        if len(filas) > 3:
            break
#     time.sleep(15)
    
#     # Extraemos el HTML de la tabla entera
#     tabla_html = driver.find_element(By.CSS_SELECTOR, '#frm1\:j_idt260').get_attribute("outerHTML")
    
#     # Creamos un objeto BeautifulSoup para analizar el html
#     soup = BeautifulSoup(tabla_html, "html.parser")
    
    
    # Extraemos columnas y filas
    headers = [th.text.strip() for th in soup.find_all('th')]
    rows = []
    for tr in soup.find_all('tr'):
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)
    
    # Creamos dataframe para ver como salieron los datos con las columnas y filas
    df_temp = pd.DataFrame(rows, columns=headers)
    print(df_temp.shape)
    
    columnas_dataframe = set(df_temp.columns)
    columnas_de_interes = set(columnas_de_interes)
    # Revisamos si el el daframe cuenta con las columnas que queremos
    columnas_faltantes = columnas_de_interes - columnas_dataframe
    print(columnas_faltantes)
    
    if not columnas_faltantes:
        df_temp = df_temp[columnas_final]
    else:
        for columna in columnas_faltantes:
            df_temp[columna] = np.nan
        df_temp = df_temp[columnas_final]
    
    lista_filas.extend(df_temp[columnas_final].to_numpy().tolist())

    # Cuando ya se genero la tabla y extrajo los datos, le doy al boton volver para que me regrese a la pagina para volver a llenar los filtro
    boton_volver = driver.find_element(By.CSS_SELECTOR, '#frm1\:j_idt268')
    boton_volver.click()

driver.quit()

In [ ]:
# Con todas las filas extraidas, genereamos el dataframe final donde almacenara todo
df_final = pd.DataFrame(lista_filas, columns=columnas_final)
df_final